## This compares to DoubleBracketIteration whenever possible

In [1]:
import inspect
import sys
sys.path.append("../../tests")
from test_models_dbi import *
def print_function_source_code( func ):
    out = inspect.getsourcelines(func)  
    from functools import reduce
    print(reduce(str.__add__, out[0]))

In [2]:
t_step =0.1
eps = 1e-2
import qibo
backend = qibo.backends.construct_backend("numpy")
qibo.set_backend("numpy")
nqubits = 3

[Qibo 0.2.7|INFO|2024-04-05 13:18:37]: Using numpy backend on /CPU:0


In [3]:
from numpy.linalg import norm
h_x = SymbolicHamiltonian( symbols.X(0) + symbols.Z(0) * symbols.X(1) + symbols.Y(2) 
                          + symbols.Y(1) * symbols.Y(2), nqubits = 3 )
d_0 = SymbolicHamiltonian(symbols.Z(0), nqubits = 3 )
h_input = h_x + d_0 

dbi = DoubleBracketIteration(deepcopy(h_input.dense))
dbi.mode = DoubleBracketGeneratorType.single_commutator


dbi2 = DoubleBracketIteration(deepcopy(h_input.dense))
dbi2.mode = DoubleBracketGeneratorType.group_commutator

V_dbi = dbi.eval_dbr_unitary(t_step, d=d_0.dense.matrix)
R_dbi = dbi2.eval_dbr_unitary(t_step, d=d_0.dense.matrix)

evolution_oracle = EvolutionOracle(h_input, "ZX",
                    mode_evolution_oracle = EvolutionOracleType.numerical)    

evolution_oracle_diagonal_target =  EvolutionOracle(d_0, "D0",
           mode_evolution_oracle = EvolutionOracleType.numerical)

gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(evolution_oracle ))
#gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator

unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step), evolution_oracle_diagonal_target )   
Q_gci = unitary_gc_from_oracles['forwards']

assert norm(Q_gci.conj().T - unitary_gc_from_oracles['backwards']) < 1e-12

h0_norm = np.linalg.norm(h_x.dense.matrix)
d0_norm = np.linalg.norm(d_0.dense.matrix)
assert norm(V_dbi - R_dbi) < 2 *t_step**1.49 * ( h0_norm + d0_norm ) * h0_norm * d0_norm

[Qibo 0.2.7|WARNING|2024-04-05 13:18:37]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-04-05 13:18:37]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.7|WARNING|2024-04-05 13:18:37]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


In [4]:
norm(Q_gci.conj().T - R_dbi )

1.9045740584321311

In [5]:
norm(Q_gci - R_dbi )

1.051669878291704

In [6]:
from scipy.linalg import expm

s_step = np.sqrt(t_step)
Vh = expm(1j * s_step * h_input.dense.matrix )

Vd = expm(1j * s_step * d_0.dense.matrix )

In [7]:
norm( Vh @ Vd @ Vh.conj().T @ Vd.conj().T - R_dbi)

5.716110830818201e-16

In [8]:
norm( Vh @ Vd @ Vh.conj().T @ Vd.conj().T - Q_gci)

1.0516698782917042

In [9]:
norm( Vh - evolution_oracle.circuit(-s_step))

0.0

In [10]:
norm( Vd - evolution_oracle_diagonal_target.circuit(-s_step))

0.0

In [17]:
norm( Vd.conj().T - evolution_oracle_diagonal_target.circuit(s_step))

0.0

In [18]:
norm( Vh.conj().T - evolution_oracle.circuit(s_step))

5.103534121246274e-16

In [13]:
from functools import reduce
S = reduce(np.ndarray.__matmul__,[ Vh,Vd,Vh.conj().T, Vd.conj().T])

In [19]:
norm(S- Q_gci)

1.904574058432131

In [15]:
norm(S- R_dbi)

5.716110830818201e-16

In [16]:
norm( Vh - gci.iterated_hamiltonian_evolution_oracle.circuit(-s_step))

0.0